# Modelo de rendimiento en las pruebas Saber 11 2017-2019

## Cargar los modulos necesarios y los datos

In [ ]:
# Importar modulos para el tratamiento de los datos y visualizaciones
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Cargar los datos, se puede hacer desde una URL
# url = 'https://gitlab.com/jairo.acosta/saber11/-/raw/main/IGAE.csv'
df = pd.read_csv('C:/Users/jacos/Desktop/IGAE.csv', header=0)

## Formatear los datos

In [ ]:
# Convertir la columna clase a valor numérico, observe que los valores antiguos
# Tienen una comilla simple de origen
df['RENDIMIENTO'] = df['RENDIMIENTO'].map(
    {"'Minimo o insuficiente'": 0, "'Satisfactorio o avanzado'": 1})

In [ ]:
# Eliminar las comillas innecesarios en las otras columnas del dataframe
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].str.replace("'",'')

In [ ]:
df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].str.replace("'",'')

In [ ]:
df['COLE_DEPTO_UBICACION'] = df['COLE_DEPTO_UBICACION'].str.replace("'",'')

In [ ]:
df['COLE_NATURALEZA'] = df['COLE_NATURALEZA'].str.replace("'",'')

In [ ]:
df['FAMI_NUMLIBROS'] = df['FAMI_NUMLIBROS'].str.replace("'",'')

In [ ]:
df['FAMI_COMELECHEDERIVADOS'] = df['FAMI_COMELECHEDERIVADOS'].str.replace("'",'')

In [ ]:
df['FAMI_COMECARNEPESCADOHUEVO'] = df['FAMI_COMECARNEPESCADOHUEVO'].str.replace("'",'')

In [ ]:
df['ESTU_DEDICACIONINTERNET'] = df['ESTU_DEDICACIONINTERNET'].str.replace("'",'')

In [ ]:
df['FAMI_PERSONASHOGAR'] = df['FAMI_PERSONASHOGAR'].str.replace("'",'')

In [ ]:
df['ESTU_DEDICACIONLECTURADIARIA'] = df['ESTU_DEDICACIONLECTURADIARIA'].str.replace("'",'')

In [ ]:
df['EDAD'] = df['EDAD'].str.replace("'",'')

In [ ]:
# Revisar los efectos del formateo de texto
df.tail()

In [ ]:
df.info()

## Crear las variables dummy

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Eliminar las variables que no generan información al modelo
del df['FAMI_EDUCACIONMADRE_No aplica']

In [ ]:
del df['FAMI_EDUCACIONPADRE_No aplica']

## Asignar valores de X e y

In [ ]:
X = df.loc[:, df.columns!= 'RENDIMIENTO']

In [ ]:
y = df['RENDIMIENTO']

## Crear el conjunto de entrenamiento y validación

In [ ]:
# Cargar el modulo para dividir el conjunto de datos en conjunto de entrenamiento y validación
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape, X_test.shape

## Modelo de regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistica = LogisticRegression()

### Ajustar el modelo a los datos

In [ ]:
logistica.fit(X_train, y_train)

### Parámetros $\theta$ del modelo

In [ ]:
logistica.intercept_

In [ ]:
logistica.coef_

### Predicción en el conjunto de validación

In [ ]:
logistica_pred = logistica.predict(X_test)

In [ ]:
logistica_pred

### Métricas del modelo

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

In [ ]:
confusion_matrix(y_test, logistica_pred)

In [ ]:
accuracy_score(y_test, logistica_pred)

In [ ]:
roc_auc_score(y_test, logistica_pred)

## Modelo de árboles

In [ ]:
from sklearn import tree

In [ ]:
arbol = tree.DecisionTreeClassifier(max_depth = 4)

### Ajustar el modelo a los datos

In [ ]:
arbol.fit(X_train, y_train)

### Predicción en el conjunto de validación

In [ ]:
arbol_pred = arbol.predict(X_test)

In [ ]:
arbol_pred

### Métricas del modelo

In [ ]:
confusion_matrix(y_test, arbol_pred)

In [ ]:
accuracy_score(y_test, arbol_pred)

In [ ]:
roc_auc_score(y_test, arbol_pred)

### Gráfica del árbol

In [ ]:
datos_graf = tree.export_graphviz(arbol, out_file = None, feature_names = X_train.columns,
                                  filled = True, class_names = ['Minimo o insuficiente','Satisfactorio o avanzado'])

In [ ]:
from IPython.display import Image
import pydotplus

In [ ]:
graph = pydotplus.graph_from_dot_data(datos_graf)
Image(graph.create_png())

In [ ]:
# Guardar como un archivo
# graph.write_png('arboloriginalclass.png')

## Modelo de Perceptrón Multicapa

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
perceptron = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(5, 5), random_state=123, max_iter=10000, activation = 'logistic')

### Transformación de los datos

In [ ]:
# Como es un modelo más complicado se escala el vector de características
# para que el modelo se ajuste a los datos de forma más ágil
from sklearn import preprocessing

In [ ]:
escala = preprocessing.StandardScaler().fit(X_train)

In [ ]:
X_esc = escala.transform(X_train)

In [ ]:
X_esc_test = escala.transform(X_test)

### Ajustar el modelo a los datos

In [ ]:
perceptron.fit(X_esc, y_train)

### Predicción en el conjunto de validación

In [ ]:
perceptron_pred = perceptron.predict(X_esc_test)

In [ ]:
perceptron_pred

### Métricas del modelo

In [ ]:
confusion_matrix(y_test, perceptron_pred)

In [ ]:
accuracy_score(y_test, perceptron_pred)

In [ ]:
roc_auc_score(y_test,perceptron_pred)